In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# 1. Load the Data
data = pd.read_csv('realistic_rice_seedling_data.csv')

In [ ]:
# Features: Environmental conditions
features = data[['Temperature (°C)', 'Humidity (%)', 'Light Intensity (lux)', 
                 'CO2 Levels (ppm)', 'Water Supply (L/day)', 
                 'Nutrient Concentration (%)', 'Soil Moisture (%)']]

In [ ]:
targets = pd.DataFrame({
    'Optimal Temperature (°C)': data['Temperature (°C)'] + np.random.uniform(0, 2, len(data)),
    'Optimal Humidity (%)': data['Humidity (%)'] + np.random.uniform(-5, 5, len(data)),
    'Optimal Light Intensity (lux)': data['Light Intensity (lux)'] + np.random.uniform(-500, 500, len(data)),
    'Optimal CO2 Levels (ppm)': data['CO2 Levels (ppm)'] + np.random.uniform(-20, 20, len(data)),
    'Optimal Water Supply (L/day)': data['Water Supply (L/day)'] + np.random.uniform(-0.1, 0.1, len(data)),
    'Optimal Nutrient Concentration (%)': data['Nutrient Concentration (%)'] + np.random.uniform(-0.5, 0.5, len(data)),
    'Optimal Soil Moisture (%)': data['Soil Moisture (%)'] + np.random.uniform(-5, 5, len(data))
})

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)
scaled_targets = scaler.fit_transform(targets)

In [ ]:
# Reshape for LSTM [samples, time steps, features]
X = np.reshape(scaled_features, (scaled_features.shape[0], 1, scaled_features.shape[1]))
y = scaled_targets

In [ ]:
# 3. Build the LSTM Model
model = Sequential()
model.add(LSTM(50, return_sequences=False, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))  # Dropout for regularization
model.add(Dense(64, activation='relu'))  # Hidden layer
model.add(Dense(y.shape[1], activation='linear'))  # Output layer: one node for each optimal condition

In [ ]:
from tensorflow.keras.optimizers import Adam
# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')


In [ ]:
# 4. Train the Model
history = model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2, verbose=1)


In [ ]:
# 5. Make Predictions for Optimization
predicted_conditions = model.predict(X)


In [ ]:
# Rescale the predictions back to original scale
predicted_conditions = scaler.inverse_transform(predicted_conditions)


In [ ]:
# Display predicted optimal conditions for first few rows
print("Predicted Optimal Conditions:")
print(predicted_conditions[:5])
